# Image Compression using Singular Value Decomposition

### Background

In this notebook, I am performing image compression through a wonderful concept called Singular Value Decomposition (SVD). I came across this concept recently in my Applied Matrix Theory course. Ever since I understood the concept and it's applications, I have been itching to put it into practice. 

In this notebook, I am going to compress both Black & White and Color images to show the subtle differences that come with compressing color images. I am also making it interactive so that you can choose different pictures from the catalog or upload your own to play with. 

Before we jump into the code, I will try to briefly explain the concept of SVD and how it can be used to compress images.

### SVD

A matrix of size m × n is a grid of real numbers consisting of m rows and n columns. A singular value decomposition (SVD) of an
m × n matrix A expresses the matrix as the product of three “simple” matrices:

A = USV

where:\
U is an m × m orthogonal matrix;\
V is an n × n orthogonal matrix;\
S is an m × n diagonal matrix with nonnegative entries, and with the diagonal entries sorted from high to low (as one goes “northwest” to “southeast).”

The columns of U are called the left singular vectors of A (these are m-vectors). The rows of V are the right singular vectors of A (these are n-vectors). The entries of S are the singular values of A. Thus with each singular vector (left or right) there is an associated singular value. The “first” or “top” singular vector refers to the one associated with the largest singular value, and so on.

### Using SVD for Image Compression

We can decompose a given image into the three color channels red, green and blue. Each channel can be represented as a 
(m × n) matrix with values ranging from 0 to 255. We can then compress the matrix A by compressing each one of the channels and then combining them.

To do this, we compute an approximation to each channel in matrix A that takes only a fraction of the space taken by that particular channel of A. Now here's the great thing about SVD: the data in the matrices U, S and V is sorted by how much it contributes to the matrix A in the product. This enables us to get quite a good approximation by simply using only the most important parts of the matrices. We now choose the top k singular values that we are going to use for the approximation. The higher this number, the better the quality of the approximation gets but also the more data is needed to encode it. This is the gist of how SVD can be used for image compression.

But note that, for image compression, more sophisticated methods like JPG that take human perception into account generally outperform compression using SVD


### Credits 
https://web.stanford.edu/class/cs168/l/l9.pdf \
http://timbaumann.info/svd-image-compression-demo/

## Section 1: Black and White Images

In [1]:
# Importing the necessary libraries

from skimage import io
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, interactive, interact_manual
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [2]:
# Reading in the images in grayscale. Using the same set of images for both Black and White and Color image compression

bw_images = {
    "Bird" : io.imread('./images/Bird.jpg', as_gray=True),
    "Chandelier" : io.imread('./images/Chandelier.jpg', as_gray=True),
    "Flowers" : io.imread('./images/Flowers.jpg', as_gray=True),
    "Tree" : io.imread('./images/Tree.jpg', as_gray=True),
    "Waterfall" : io.imread('./images/Waterfall.jpg', as_gray=True)
}

In [3]:
# Showing image in widgets

def show_bw_images(Image_Name):
    fig = plt.figure(figsize=(18, 18))
    plt.title("Image Name: "+Image_Name+"\n")
    plt.imshow(bw_images[Image_Name], cmap = plt.cm.gray)  # picking the right color scale for display 
    plt.axis('off')
    plt.show()

In [4]:
interact(show_bw_images, Image_Name = list(bw_images.keys()))

interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.show_bw_images(Image_Name)>

In [6]:
# Checking the dimensions of each image to give an idea of how many singular values are present

for image in bw_images.keys():
    print(f"{image} - {bw_images[image].shape}")

Bird - (5184, 3456)
Chandelier - (3415, 5122)
Flowers - (7329, 5497)
Tree - (6000, 4000)
Waterfall - (3334, 5000)


In [8]:
def compress_bw_image(Image_Name, k): 
    
    fig = plt.figure(figsize=(18, 18))
    
    # Showing the original image
    fig.add_subplot(1, 2, 1)
    plt.title(f"Original Image - {Image_Name}")
    plt.imshow(bw_images[Image_Name], cmap = plt.cm.gray)
    plt.axis('off')
    
    # Performing SVD and the compression
    U, S, V = np.linalg.svd(bw_images[Image_Name], full_matrices = True)
    u = U[:, 0:k]
    v = V[0:k]
    compressed_image = u @ np.diag(S[0:k]) @ v
    
    # Showing the compressed image
    fig.add_subplot(1, 2, 2)
    plt.title(f"Compressed Image - {Image_Name} (Top {k} Singular values)")
    plt.imshow(compressed_image, cmap = plt.cm.gray)
    plt.axis('off')
    plt.show()
    
    # Showing the compression ratio
    dim1 = bw_images[Image_Name].shape
    size1 = dim1[0]*dim1[1]
    print(f"Original Image Size (in pixels)- {size1}")
    size2 = min(dim1)*k + k + k*max(dim1)
    print(f"Compressed Image Size (in pixels)- {size2}")
    ratio = round(size1/size2,2)
    print(f"Compression Ratio - {ratio}")    

In [9]:
interact(compress_bw_image, Image_Name = list(bw_images.keys()), k = (0,600))

interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.compress_bw_image(Image_Name, k)>

## Section 2: Color Images

In [10]:
# This time, we'll be reading in the images normally. Using the same set of images for both Black and White and Color image compression

color_images = {
    "Bird" : io.imread('./images/Bird.jpg'),
    "Chandelier" : io.imread('./images/Chandelier.jpg'),
    "Flowers" : io.imread('./images/Flowers.jpg'),
    "Tree" : io.imread('./images/Tree.jpg'),
    "Waterfall" : io.imread('./images/Waterfall.jpg')
}

In [11]:
# Showing image in widgets

def show_color_images(Image_Name):
    fig = plt.figure(figsize=(18, 18))
    plt.title("Image Name: "+Image_Name+"\n")
    plt.imshow(color_images[Image_Name])  # picking the right color scale for display 
    plt.axis('off')
    plt.show()

In [12]:
interact(show_color_images, Image_Name = list(color_images.keys()))

interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.show_color_images(Image_Name)>

In [13]:
# Checking the dimensions of each image to give an idea of how many singular values are present

for image in color_images.keys():
    print(f"{image} - {color_images[image].shape}")

Bird - (5184, 3456, 3)
Chandelier - (3415, 5122, 3)
Flowers - (7329, 5497, 3)
Tree - (6000, 4000, 3)
Waterfall - (3334, 5000, 3)


Here you can see that there's a difference in the number of dimensions when compared to the Black and White images. Now you have 3 dimensions instead of 2, where the third dimension contains the color channel for the image (RGB). So, in order to perform compression, we need to do it individually for each color channel and then finally combine it.

In [14]:
def compress_color_image(Image_Name, k): 
        
    fig = plt.figure(figsize=(18, 18))
    
    # Showing the original image
    fig.add_subplot(1, 2, 1)
    plt.title(f"Original Image - {Image_Name}")
    plt.imshow(color_images[Image_Name])
    plt.axis('off')
    
    # Breaking the 3d matrix into 3 2d matrices - 1 for each color channel
    r = color_images[Image_Name][:,:,0]
    g = color_images[Image_Name][:,:,1]
    b = color_images[Image_Name][:,:,2]
    
    # Performing SVD for each matrix
    Ur, Sr, Vr = np.linalg.svd(r, full_matrices = True)
    Ug, Sg, Vg = np.linalg.svd(g, full_matrices = True)
    Ub, Sb, Vb = np.linalg.svd(b, full_matrices = True)
    
    # Compressing all the matrices individually
    ur = Ur[:, 0:k]
    vr = Vr[0:k]
    
    ug = Ug[:, 0:k]
    vg = Vg[0:k]
    
    ub = Ub[:, 0:k]
    vb = Vb[0:k]
    
    compressed_image_red = ur @ np.diag(Sr[0:k]) @ vr
    compressed_image_green = ug @ np.diag(Sg[0:k]) @ vg
    compressed_image_blue = ub @ np.diag(Sb[0:k]) @ vb
    
    # Combining the three 2D matrices into a single 3D matrix
    compressed_image = np.zeros(color_images[Image_Name].shape)
    compressed_image[:,:,0] = compressed_image_red
    compressed_image[:,:,1] = compressed_image_green
    compressed_image[:,:,2] = compressed_image_blue
    
    # Converting the values in the matrix to be in the range [0,255]
    compressed_image_final = np.array(compressed_image/np.amax(compressed_image)*255, np.int32)
  
    # Showing the compressed image
    fig.add_subplot(1, 2, 2)
    plt.title(f"Compressed Image - {Image_Name} (Top {k} Singular values)")
    plt.imshow(compressed_image_final.astype('uint8'))
    plt.axis('off')
    plt.show()
    
    # Showing the compression ratio
    dim1 = color_images[Image_Name].shape
    size1 = dim1[0]*dim1[1]
    print(f"Original Image Size (in pixels)- {size1}")
    size2 = min(dim1)*k + k + k*max(dim1)
    print(f"Compressed Image Size (in pixels)- {size2}")
    ratio = round(size1/size2,2)
    print(f"Compression Ratio - {ratio}")    

In [15]:
interact(compress_color_image, Image_Name = list(color_images.keys()), k = (0,600))

interactive(children=(Dropdown(description='Image_Name', options=('Bird', 'Chandelier', 'Flowers', 'Tree', 'Wa…

<function __main__.compress_color_image(Image_Name, k)>

The difference between original and compressed images is more apparent for lower rank approximations for colored images than for Black and White images. But neverthless, using SVD, we can compress the image considerably by picking the top features of the image (top k singular values) without noticing visible changes in the quality of the image.